## Problem 7.1: Writing your own MCMC sampler

In [1]:
import itertools

import numpy as np
import pandas as pd
import scipy.stats as st
import random

import numba

import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
mu = np.array([10.0, 20])
cov = np.array([[4, -2],[-2, 6]])
inv_cov = np.linalg.inv(cov)

@numba.jit(nopython=True)
def log_test_distribution(x, mu, inv_cov):
    """
    Unnormalized log posterior of a multivariate Gaussian.
    """
    return -np.dot((x-mu), np.dot(inv_cov, (x-mu))) / 2

In [3]:
def mh_step(x, logpost, logpost_current, sigma, args=()):
    """
    Parameters
    ----------
    x : ndarray, shape (n_variables,)
        The present location of the walker in parameter space.
    logpost : function
        The function to compute the log posterior. It has call
        signature `logpost(x, *args)`.
    logpost_current : float
        The current value of the log posterior.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `logpost()` function.

    Returns
    -------
    output : ndarray, shape (n_variables,)
        The position of the walker after the Metropolis-Hastings
        step. If no step is taken, returns the inputted `x`.
    """
    # Get next step
    x_next = np.random.multivariate_normal(x, sigma)

    # Calculate r
    theta_p = np.exp(logpost(x_next, *args))
    theta_i = np.exp(logpost_current)
    r = theta_p / theta_i
#     print(r)
    
    # Choose to accept or reject step    
    p = np.random.uniform(0, 1)
    if r >= 1:
        return x_next, 1
    elif p <= r:
        return x_next, 1
    else:
        return x, 0

In [15]:
def mh_sample(logpost, x0, sigma, args=(), n_burn=1000, n_steps=1000,
              variable_names=None):
    """
    Parameters
    ----------
    logpost : function
        The function to compute the log posterior. It has call
        signature `logpost(x, *args)`.
    x0 : ndarray, shape (n_variables,)
        The starting location of a walker in parameter space.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `logpost()` function.
    n_burn : int, default 1000
        Number of burn-in steps.
    n_steps : int, default 1000
        Number of steps to take after burn-in.
    variable_names : list, length n_variables
        List of names of variables. If None, then variable names
        are sequential integers.
    
    Returns
    -------
    output : DataFrame
        The first `n_variables` columns contain the samples.
        Additionally, column 'lnprob' has the log posterior value
        at each sample.
    """
    x = x0
    mu, inv_cov = args
    n_accept = 0
    
    # Steps that will be burned
    for i in range(n_burn):
        logpost_current = logpost(x, *args)
        x, accept = mh_step(x, logpost, logpost_current, sigma, args=(mu, inv_cov))
        
    # Set up empty arrays to store info
    n_variables = []
    lnprob = []
    
    # Step
    for i in range(n_steps):
        n_variables.append(x)
        lnprob.append(logpost_current)
        
        logpost_current = logpost(x, *args)
        x, accept = mh_step(x, logpost, logpost_current, sigma, args=(mu, inv_cov))
        n_accept += accept
        
    df = pd.DataFrame(data=n_variables, columns=['x', 'y'])
    df['lnprob'] = lnprob
    
    return df, n_accept/n_steps

### can r be greater than 1?

add a function to tune sigma if acceptance rate is not around 0.4. but this is inefficient because it still calculates all the burn and finds all the samples before returning the acceptance rate. so we can still improve on this

In [19]:
def tune_sigma(accept_rate, sigma):
    if accept_rate < 0.001:
        return sigma * 0.1
    elif accept_rate < 0.05:
        return sigma * 0.5
    elif accept_rate < 0.2:
        return sigma * 0.9
    elif accept_rate > 0.5:
        return sigma * 1.1
    elif accept_rate > 0.75:
        return sigma * 2
    elif accept_rate > 0.95:
        return sigma * 10
    else:
        return sigma

In [30]:
# Choose arbitrary x0 and sigma
x0 = np.array([10, 5])
sigma = np.array([[100, -3],[-3, 100]])

# Take samples
df_samples, accept_rate = mh_sample(log_test_distribution, x0, sigma, args=(mu, inv_cov), n_burn=1000, n_steps=5000, variable_names=None)
while accept_rate < 0.2 or accept_rate > 0.5:
    sigma = tune_sigma(accept_rate, sigma)
    print('new sigma:', sigma)
    df_samples, accept_rate = mh_sample(log_test_distribution, x0, sigma, args=(mu, inv_cov), n_burn=1000, n_steps=5000, variable_names=None)
    print('current acceptance rate:', accept_rate)
    
# Take a look
df_samples.head()

new sigma: [[90.  -2.7]
 [-2.7 90. ]]
current acceptance rate: 0.0826
new sigma: [[81.   -2.43]
 [-2.43 81.  ]]
current acceptance rate: 0.0964
new sigma: [[72.9   -2.187]
 [-2.187 72.9  ]]
current acceptance rate: 0.1042
new sigma: [[65.61   -1.9683]
 [-1.9683 65.61  ]]
current acceptance rate: 0.1164
new sigma: [[59.049   -1.77147]
 [-1.77147 59.049  ]]
current acceptance rate: 0.1262
new sigma: [[53.1441   -1.594323]
 [-1.594323 53.1441  ]]
current acceptance rate: 0.1284
new sigma: [[47.82969   -1.4348907]
 [-1.4348907 47.82969  ]]
current acceptance rate: 0.1474
new sigma: [[43.046721   -1.29140163]
 [-1.29140163 43.046721  ]]
current acceptance rate: 0.1516
new sigma: [[38.7420489  -1.16226147]
 [-1.16226147 38.7420489 ]]
current acceptance rate: 0.1734
new sigma: [[34.86784401 -1.04603532]
 [-1.04603532 34.86784401]]
current acceptance rate: 0.1858
new sigma: [[31.38105961 -0.94143179]
 [-0.94143179 31.38105961]]
current acceptance rate: 0.1958
new sigma: [[28.24295365 -0.847288

,x,y,lnprob
0,12.411611,17.940924,-0.79979
1,12.411611,17.940924,-0.79979
2,7.027952,21.532542,-0.79979
3,7.027952,21.532542,-1.10435
4,7.027952,21.532542,-1.10435


Now let's plot to check.

In [31]:
# Plot
p = bokeh.plotting.figure(width=400, height=400,
                          x_axis_label='x', 
                          y_axis_label='y')

# Plot samples
p.circle(df_samples['x'], df_samples['y'], alpha=0.025)

# Overlay multivariate gaussian
x, y = np.random.multivariate_normal(mu, cov, 5000).T
p.circle(x, y, alpha=0.025, color='orange')
bokeh.io.show(p)

Looks like our samples in blue match the multivariate gaussian distribution in orange.

Check that the covariance of our samples is similar to the inputted covariance.

In [32]:
np.cov([df_samples['x'], df_samples['y']])

array([[ 4.00109112, -2.04779239],
       [-2.04779239,  6.41828623]])

Yes it's similar :)

Trying to plot the corner plot?

In [33]:
df_samples['divergent__'] = 0

In [34]:
bokeh.io.show(bebi103.viz.corner(df_samples, pars=['x', 'y']))